In [135]:

import pandas as pd
import numpy as np
# pd.set_option()
import matplotlib.pyplot as plt


pd.set_option('display.max_columns',50)

train = pd.read_csv('./Data/Yes_ml2_train.csv')

# print(train.shape,'\n')
# print(train.info(),'\n')
# print(train.describe().T,'\n')
# print(train.head(20),'\n')
# print(train.tail(20),'\n')

In [136]:
train = train[['Serial Number','2010 Deposits','2011 Deposits','2012 Deposits', '2013 Deposits', '2014 Deposits',
       '2015 Deposits', '2016 Deposits']]

In [137]:
from sklearn.preprocessing import Imputer
mean_imputer = Imputer()
median_imputer = Imputer(strategy='median')


In [138]:
cols = ['2010 Deposits','2011 Deposits','2012 Deposits', '2013 Deposits', '2014 Deposits',
       '2015 Deposits']
for col in cols:
    train[col]  = mean_imputer.fit_transform(train[[col]]).ravel()
# train['log_2015'].impute('mean')

# train['log_2014'].describe()

In [139]:
# logs= ['log_2015', 'log_2014', 'log_2012', 'log_2013']

for log in cols :
    iqr = train[log].quantile(0.75) - train[log].quantile(0.25)
# print(iqr)
    start = train[log].quantile(0.25) - 1.5*iqr
    end = train[log].quantile(0.75) + 1.5*iqr
# print(start)
# print(end)
    train = train.loc[(start < train[log]) & (train[log]< end)]  

In [140]:
print(train.shape)
train.describe()

(3436, 8)


,Serial Number,2010 Deposits,2011 Deposits,2012 Deposits,2013 Deposits,2014 Deposits,2015 Deposits,2016 Deposits
count,3436.000000,3436.000000,3436.000000,3436.000000,3436.000000,3436.000000,3436.000000,3436.000000
mean,1942.200524,53711.883513,53311.023118,49803.121558,49528.695844,51098.466643,56223.172584,65429.800349
std,1108.569564,38780.542493,40611.841731,37466.555181,35644.171205,34283.068170,35509.192187,40296.586786
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,942.750000,20926.500000,21000.000000,20818.750000,22198.000000,24661.000000,28869.500000,35361.000000
50%,2034.500000,41462.500000,40136.500000,39437.000000,40282.000000,43078.500000,48237.500000,56372.500000
75%,2906.250000,91551.250000,80282.250000,70923.500000,69885.750000,71765.500000,77922.750000,88527.000000
max,3772.000000,217790.000000,139699.000000,142428.000000,145246.000000,148199.000000,155387.000000,247389.000000


In [82]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [100]:
stdScaler = StandardScaler()
mnScaler = MinMaxScaler()

In [62]:
train_d = stdScaler.fit(train)

In [63]:
train_d

StandardScaler(copy=True, with_mean=True, with_std=True)

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [141]:
train_select = train[['2010 Deposits','2011 Deposits','2012 Deposits', '2013 Deposits', '2014 Deposits','2015 Deposits', '2016 Deposits']]

In [142]:
target = train_select['2016 Deposits']
predictors = train_select.drop(['2016 Deposits'],axis=1)

In [143]:
predictors.isnull().sum()

2010 Deposits    0
2011 Deposits    0
2012 Deposits    0
2013 Deposits    0
2014 Deposits    0
2015 Deposits    0
dtype: int64

In [144]:
X_train, X_test, Y_train, Y_test = train_test_split(predictors, target, train_size=0.8,test_size=0.2, random_state=7)

In [110]:
X_train = stdScaler.fit_transform(X_train)
X_test = stdScaler.transform(X_test)

# X_train = mnScaler.fit_transform(X_train)
# X_test = mnScaler.transform(X_test)

In [145]:
from sklearn import linear_model, svm, tree, ensemble

models = [
    linear_model.LinearRegression(),
    linear_model.Ridge(),
    linear_model.Lasso(),
    linear_model.ElasticNet(),
    linear_model.BayesianRidge(),
#     linear_model.RANSACRegressor(),
    svm.LinearSVR(),
    svm.SVR(),
    tree.DecisionTreeRegressor(),
    tree.ExtraTreeRegressor(),
    ensemble.RandomForestRegressor(),
    ensemble.GradientBoostingRegressor()

]

In [146]:
for model in models:
    model.fit(X_train,Y_train)
    print('-'*40)
    print(model.__class__.__name__)
    
    train_pred = model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(Y_train, train_pred))
    print("Root mean square error train {}".format(train_rmse))    

    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(Y_test, pred))
    print("Root mean square error test {} ".format(rmse))

----------------------------------------
LinearRegression
Root mean square error train 10980.864579066692
Root mean square error test 8575.360531712207 
----------------------------------------
Ridge
Root mean square error train 10980.864579066692
Root mean square error test 8575.36053171341 
----------------------------------------
Lasso
Root mean square error train 10980.864579066694
Root mean square error test 8575.360538319434 
----------------------------------------
ElasticNet
Root mean square error train 10980.864579066692
Root mean square error test 8575.360536657474 
----------------------------------------
BayesianRidge
Root mean square error train 10980.874316061518
Root mean square error test 8576.097754073133 
----------------------------------------
LinearSVR
Root mean square error train 11270.61998297552
Root mean square error test 8844.141419075544 
----------------------------------------
SVR
Root mean square error train 41266.4588510978
Root mean square error test 410

In [147]:
test = pd.read_csv('./Data/Yes_ml2_test.csv')

In [148]:
test.isnull().sum()

Serial Numer          0
Main Office           0
Branch Number         0
Established Date      0
Acquired Date       122
City                  0
County                0
State                 0
2010 Deposits         0
2011 Deposits         0
2012 Deposits         0
2013 Deposits         0
2014 Deposits         0
2015 Deposits         0
dtype: int64

In [149]:
test = test[ ['Serial Numer','2010 Deposits','2011 Deposits','2012 Deposits', '2013 Deposits', '2014 Deposits',
       '2015 Deposits']]

In [150]:
serial_numer = pd.DataFrame(test['Serial Numer'])
test = test[['2010 Deposits','2011 Deposits','2012 Deposits', '2013 Deposits', '2014 Deposits','2015 Deposits']]

In [151]:
for i in range(11):
    test_pred = models[i].predict(test)   
    result = pd.concat([serial_numer['Serial Numer'],pd.Series(test_pred, name='Deposits 2016')], axis =1,)
    pd.DataFrame.to_csv(result,path_or_buf='./output/resultsML2_yes_{}.csv'.format(i),index=False)   